In [19]:
from flask import Flask # 플라스크 클래스 임포트
from flask import request, redirect
import pickle # 데이터 저장/로딩을 위한 모듈
import pandas as pd
import numpy as np
import math



def preprocessing(data_dict):
    with open('pro_column_name.pkl','rb') as f :
        pro_columns = pickle.load(f)
    
    df = pd.DataFrame( np.zeros((1,20)) , columns=pro_columns)
   
    df['AGE'] = int(data_dict['AGE']) 
    df['INTELLIGIBILITY'] = data_dict['INTELLIGIBILITY'] 
    df['APPLY'] = data_dict['APPLY']
    df['CLASS_'+data_dict['CLASS']] = 1 
    df['DISTANCE_'+data_dict['DISTANCE']] = 1
    df['PURPOSE_'+data_dict['PURPOSE']] = 1
    df['ROOT_'+data_dict['ROOT']] = 1
    df['SEX_'+data_dict['SEX']] = 1
    df['MAJOR_'+data_dict['MAJOR']] = 1
    
    
    
    
    
    return df


In [20]:
from flask import Flask # 플라스크 클래스 임포트
from flask import request, redirect
import pickle # 데이터 저장/로딩을 위한 모듈
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup as bs
from flask_cors import CORS
import random


app = Flask(__name__) # 내장변수 name을 이용해 서버를 구동시키는 객체 생성
app.config['JSON_AS_ASCII'] = False
CORS(app)

with open('pro_model.pkl', 'rb') as f :
    model = pickle.load(f) # pickle을 이용했기때문에 model안에는 sklearn의 결정트리 클래스가 불러와진다.

@app.route("/ML", methods=["GET",'POST']) #테스트 페이지를 만들어도 연결이됨
def predict_result() :
    if request.method == 'POST' :
        display(request.form) #사용자가 사용하고 있는 페이지에거 post 방식으로 값을넘겨준다면 form 태그 안에 값으로 넘겨주고 받는다
        # 넘어온 값을 전처리
        df = preprocessing(request.form)
        pre = math.trunc(model.predict_proba(df[:1])[:,1][0]*100)
        result = (100-pre)
        
        #flask 에서 예측값(모델 실행후의 결과)를 result.jsp 페이지로 넘긴다 예측값은 url(쿼리스트링 방식 사용)
        
        return redirect("http://localhost:8084/CommunityWeb/ML.jsp?predict="+str(result))
                         #------------------여기가 JSP port 랑 연결------------------------
    



    
        

@app.route("/crawl", methods=["GET",'POST'])
def func():
    li_index = []
    li_answerPer = []
    res = req.get('https://www.acmicpc.net/problemset/'+str(random.randint(1,13)))
    soup = bs(res.text,"lxml")

    for i in range(1,100):
        try:
            answerPer = soup.select('#problemset > tbody > tr:nth-of-type('+str(i)+') > td:nth-of-type(6)')
            index = soup.select('#problemset > tbody > tr:nth-of-type('+str(i)+') > td.list_problem_id')
            str1 = index[0].text
            str2 = answerPer[0].text.replace("%","")
    
            li_index.append(str1)
            li_answerPer.append(str2)
        except:
            pass

    unique = []

    for i in range(len(li_index)):
    
        per = li_answerPer[i]
        if float(per) > 20.00:
       
            unique.append(li_index[i])
        

    rndIndex = random.randrange(0, len(unique))
#랜덤 추출
    res2 = req.get('https://www.acmicpc.net/problem/'+str(unique[rndIndex]))
    soup2 = bs(res2.text,"lxml")
    list1 =soup2.select('body > div.wrapper > div.container.content > div.row > div:nth-of-type(3)')
    list2 =soup2.select('body > div.wrapper > div.container.content > div.row > div:nth-of-type(4)')
    list3 =soup2.select('body > div.wrapper > div.container.content > div.row > div:nth-of-type(5)')
    list4 =soup2.select('body > div.wrapper > div.container.content > div.row > div:nth-of-type(6)') 
    body=list1+list2+list3+list4

    result = ""
    for s in body:
        result += str(s)
        
# 없는 이미지 처리
    if result.find("//onlinejudgeimages.s3-ap-northeast-1.amazonaws.com/upload") == -1:
        if result.find("/JudgeOnline/upload/") > 0:
            result= result.replace("/JudgeOnline/upload/","https://onlinejudgeimages.s3-ap-northeast-1.amazonaws.com/upload/")
        else :
            result= result.replace("/upload/","https://onlinejudgeimages.s3-ap-northeast-1.amazonaws.com/upload/")

    result= result.replace('<button class="btn btn-link copy-button" data-clipboard-target="#sample-output-1" style="padding: 0px;" type="button">복사</button>','')
    result= result.replace('<button class="btn btn-link copy-button" data-clipboard-target="#sample-input-1" style="padding: 0px;" type="button">복사</button>','')
    return result


    
if __name__ == "__main__" : # .py 파일에서 main함수 역할
    app.run(host="localhost", port="9999") #jsp 액션에 이 호스트랑 포트를 넣으면 여기 보이지않은 파이썬열림 
                  

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9999/ (Press CTRL+C to quit)
